How to setup:

install aws-cli

Install boto3

How to pull data from AWS API

In [1]:
from time import time
from datetime import datetime
import pandas as pd
import boto3

client = boto3.client('ec2')

response = client.describe_spot_price_history(
    Filters=[
        {
        },
    ],
    AvailabilityZone='eu-central-1a',
    DryRun=False,
    EndTime=datetime(2023, 2, 21),
    InstanceTypes=[
        'a1.medium'
        ],
    MaxResults=123,
    ProductDescriptions=[
        'Linux/UNIX (Amazon VPC)'
    ],
    StartTime=datetime(2022, 2, 1)
)


In [2]:
response

{'NextToken': '',
 'SpotPriceHistory': [{'AvailabilityZone': 'eu-central-1a',
   'InstanceType': 'a1.medium',
   'ProductDescription': 'Linux/UNIX',
   'SpotPrice': '0.029100',
   'Timestamp': datetime.datetime(2023, 2, 20, 8, 32, 43, tzinfo=tzutc())},
  {'AvailabilityZone': 'eu-central-1a',
   'InstanceType': 'a1.medium',
   'ProductDescription': 'Linux/UNIX',
   'SpotPrice': '0.029100',
   'Timestamp': datetime.datetime(2023, 2, 19, 18, 30, 9, tzinfo=tzutc())},
  {'AvailabilityZone': 'eu-central-1a',
   'InstanceType': 'a1.medium',
   'ProductDescription': 'Linux/UNIX',
   'SpotPrice': '0.029100',
   'Timestamp': datetime.datetime(2023, 2, 18, 18, 29, 59, tzinfo=tzutc())},
  {'AvailabilityZone': 'eu-central-1a',
   'InstanceType': 'a1.medium',
   'ProductDescription': 'Linux/UNIX',
   'SpotPrice': '0.029100',
   'Timestamp': datetime.datetime(2023, 2, 17, 18, 29, 59, tzinfo=tzutc())},
  {'AvailabilityZone': 'eu-central-1a',
   'InstanceType': 'a1.medium',
   'ProductDescription': 'Li

How to pull instance spects values

In [36]:
response = client.describe_instance_types(
    DryRun=False,
    InstanceTypes=[
        'a1.medium'],
    Filters=[
        {},
    ],
)

In [37]:
response

{'InstanceTypes': [{'InstanceType': 'a1.medium',
   'CurrentGeneration': False,
   'FreeTierEligible': False,
   'SupportedUsageClasses': ['on-demand', 'spot'],
   'SupportedRootDeviceTypes': ['ebs'],
   'SupportedVirtualizationTypes': ['hvm'],
   'BareMetal': False,
   'Hypervisor': 'nitro',
   'ProcessorInfo': {'SupportedArchitectures': ['arm64'],
    'SustainedClockSpeedInGhz': 2.3},
   'VCpuInfo': {'DefaultVCpus': 1,
    'DefaultCores': 1,
    'DefaultThreadsPerCore': 1},
   'MemoryInfo': {'SizeInMiB': 2048},
   'InstanceStorageSupported': False,
   'EbsInfo': {'EbsOptimizedSupport': 'default',
    'EncryptionSupport': 'supported',
    'EbsOptimizedInfo': {'BaselineBandwidthInMbps': 300,
     'BaselineThroughputInMBps': 37.5,
     'BaselineIops': 2500,
     'MaximumBandwidthInMbps': 3500,
     'MaximumThroughputInMBps': 437.5,
     'MaximumIops': 20000},
    'NvmeSupport': 'required'},
   'NetworkInfo': {'NetworkPerformance': 'Up to 10 Gigabit',
    'MaximumNetworkInterfaces': 2,
 

How to setup:

Install az cli.

Create an azure account.

Login to az with `az login`.

Run command to create token. `az account get-access-token`

How to pull spot pricing data from Azure api

In [5]:
import requests
import json
from tabulate import tabulate 


def build_pricing_table(json_data, table_data):
    for item in json_data['Items']:
        meter = item['meterName']
        table_data.append([item['armSkuName'], item['retailPrice'], item['unitOfMeasure'], item['armRegionName'], meter, item['productName']])
        
def main():
    table_data = []
    table_data.append(['SKU', 'Retail Price', 'Unit of Measure', 'Region', 'Meter', 'Product Name'])
    
    api_url = "https://prices.azure.com/api/retail/prices?api-version=2021-10-01-preview"
    query = "armRegionName eq 'germanywestcentral' and armSkuName eq 'Standard_A1_v2' and priceType eq 'Consumption' and contains(meterName, 'Spot')"
    response = requests.get(api_url, params={'$filter': query})
    json_data = json.loads(response.text)
    
    build_pricing_table(json_data, table_data)
    nextPage = json_data['NextPageLink']
    
    while(nextPage):
        response = requests.get(nextPage)
        json_data = json.loads(response.text)
        nextPage = json_data['NextPageLink']
        build_pricing_table(json_data, table_data)

    #print(tabulate(table_data, headers='firstrow', tablefmt='psql'))
    print(json_data)

In [6]:
main()

{'BillingCurrency': 'USD', 'CustomerEntityId': 'Default', 'CustomerEntityType': 'Retail', 'Items': [{'currencyCode': 'USD', 'tierMinimumUnits': 0.0, 'retailPrice': 0.009329, 'unitPrice': 0.009329, 'armRegionName': 'germanywestcentral', 'location': 'DE West Central', 'effectiveStartDate': '2023-01-01T00:00:00Z', 'effectiveEndDate': '2023-02-28T23:59:00Z', 'meterId': '622d3d92-3b75-5ef1-a100-e35d7aad2607', 'meterName': 'A1 v2 Spot', 'productId': 'DZH318Z0BPW1', 'skuId': 'DZH318Z0BPW1/0154', 'availabilityId': None, 'productName': 'Virtual Machines Av2 Series Windows', 'skuName': 'A1 v2 Spot', 'serviceName': 'Virtual Machines', 'serviceId': 'DZH313Z7MMC8', 'serviceFamily': 'Compute', 'unitOfMeasure': '1 Hour', 'type': 'Consumption', 'isPrimaryMeterRegion': True, 'armSkuName': 'Standard_A1_v2'}, {'currencyCode': 'USD', 'tierMinimumUnits': 0.0, 'retailPrice': 0.004548, 'unitPrice': 0.004548, 'armRegionName': 'germanywestcentral', 'location': 'DE West Central', 'effectiveStartDate': '2023-01-

How to pull instance spec data from Azure api

In [15]:
def build_pricing_table(json_data, table_data):
    json
    for item in json_data['value']:
        table_data.append([item['name'], item['numberOfCores'], item['osDiskSizeInMB'], item['resourceDiskSizeInMB'], item["memoryInMB"], item['maxDataDiskCount']])
        
def main():
    table_data = []
    table_data.append(['SKUName', 'Number of Cores', 'OS Disk Size in MB', 'Resource Disk Size in MB', 'Memory in MB', 'Max Data Disk Count'])
    sub_id = "secret_sub"
    api_url = f'https://management.azure.com/subscriptions/{sub_id}/providers/Microsoft.Compute/locations/germanywestcentral/vmSizes?api-version=2022-11-01'
    query = "armRegionName eq 'germanywestcentral'"

    response = requests.get(api_url, params={'$filter': query}, headers={'Authorization': f'Bearer {TOKEN}'}
    json_data = json.loads(response.text)
    
    build_pricing_table(json_data, table_data)

    print(tabulate(table_data, headers='firstrow', tablefmt='psql'))
    #print(json_data)
    
    

In [16]:
main()

+------------------------+-------------------+----------------------+----------------------------+----------------+-----------------------+
| SKUName                |   Number of Cores |   OS Disk Size in MB |   Resource Disk Size in MB |   Memory in MB |   Max Data Disk Count |
|------------------------+-------------------+----------------------+----------------------------+----------------+-----------------------|
| Standard_A1_v2         |                 1 |              1047552 |                      10240 |           2048 |                     2 |
| Standard_A2m_v2        |                 2 |              1047552 |                      20480 |          16384 |                     4 |
| Standard_A2_v2         |                 2 |              1047552 |                      20480 |           4096 |                     4 |
| Standard_A4m_v2        |                 4 |              1047552 |                      40960 |          32768 |                     8 |
| Standard_A4_v2    

How to setup:
    
Download gcloud cli from: https://cloud.google.com/sdk/docs/install
        
Install it using the install.sh script.

Restart shell and run `gcloud init` to login. 

Select project where you have enabled public api.

Create api key. 

API_KEY = Some_Key

How to pull spot pricing data:

Price = "units" + "nanos"/10 000 000. 

In [47]:
import requests
import json
from tabulate import tabulate 
import pandas as pd

def main():
    API_KEY = "secret"
    SERVICE_ID = "6F81-5844-456A" # service ID for compute engine
    START_TIME = "2023-02-21T01:01:01Z"
    END_TIME = "2023-02-21T01:01:01Z"
    services_url = f'https://cloudbilling.googleapis.com/v1/services?key=<var>{API_KEY}</var>'
    api_url = f'https://cloudbilling.googleapis.com/v1/services/{SERVICE_ID}/skus?currencyCode=USD&startTime={START_TIME}&endTime={END_TIME}&key={API_KEY}'
    table_data = []
    
    response = requests.get(api_url)
    json_data = json.loads(response.text)
    NEXT_PAGE_TOKEN = json_data['nextPageToken']
    data = pd.DataFrame()
    df = pd.json_normalize(json_data, record_path=['skus'], max_level=1)

    
    while(NEXT_PAGE_TOKEN):
        api_url = f'https://cloudbilling.googleapis.com/v1/services/6F81-5844-456A/skus?currencyCode=USD&startTime={START_TIME}&endTime={END_TIME}&key={API_KEY}&pageToken={NEXT_PAGE_TOKEN}'
        response = requests.get(api_url)
        json_data = json.loads(response.text)
        NEXT_PAGE_TOKEN = json_data['nextPageToken']
        data = pd.concat([df, pd.json_normalize(json_data, record_path=['skus'], max_level=1)])

    print(data.info(memory_usage='deep'))
    
    # with open("test.json", "w") as outfile:
    #     json.dump(json_data, outfile)
    
    # print("Done")
    cleaned_data = data[(data['category.usageType'] == "Preemptible")]
    cleaned_data = cleaned_data[(cleaned_data['description'].str.startswith("Spot Preemptible"))]
    cleaned_data = cleaned_data[(~cleaned_data['description'].str.contains("Custom"))]
    cleaned_data['description'] = cleaned_data['description'].str[17:].str.strip()
    
    ## remove "running in XXXX"
    ## add location category
#     print(cleaned_data['category.usageType'].head(10))
#     print(cleaned_data.info(memory_usage='deep'))
    
    print(cleaned_data["description"])
    print(data.info(memory_usage='deep'))
    
    
    
    

How to pull spec data:

In [48]:
main()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5028 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   name                         5028 non-null   object
 1   skuId                        5028 non-null   object
 2   description                  5028 non-null   object
 3   serviceRegions               5028 non-null   object
 4   pricingInfo                  5028 non-null   object
 5   serviceProviderName          5028 non-null   object
 6   category.serviceDisplayName  5028 non-null   object
 7   category.resourceFamily      5028 non-null   object
 8   category.resourceGroup       5028 non-null   object
 9   category.usageType           5028 non-null   object
 10  geoTaxonomy.type             3646 non-null   object
 11  geoTaxonomy.regions          3646 non-null   object
dtypes: object(12)
memory usage: 4.3 MB
None
2               Compute optimized Ram running in Mon

In [26]:
str = "Spot Preemptible Compute optimized Ram running"

print(str[17:]) 

Compute optimized Ram running
